In [1]:
from sklearn.datasets import load_iris
from CART import DecisionTree
from sklearn.datasets import fetch_california_housing
from CART import train_test_split_data, evaluate_classification, evaluate_regression

#### 房价回归树

In [2]:
# Load California Housing dataset
california = fetch_california_housing()
X = california.data
y = california.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split_data(X, y, test_size=0.2, random_state=42)

# Initialize and train the decision tree for regression
reg_tree = DecisionTree(task='regression', min_samples_split=2, min_impurity_decrease=1e-7)
reg_tree.fit(X_train, y_train)

# Make predictions
y_pred = reg_tree.predict(X_test)

# Evaluate
mse = evaluate_regression(y_test, y_pred)
print(f"California Housing Regression MSE: {mse:.4f}")


California Housing Regression MSE: 0.5036


#### iris分类树

In [3]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split_data(X, y, test_size=0.2, random_state=42)

# Initialize and train the decision tree for classification
clf_tree = DecisionTree(task='classification', min_samples_split=2, min_impurity_decrease=1e-7)
clf_tree.fit(X_train, y_train)

# 打印决策树结构
print("Decision Tree Structure:")
clf_tree.print_tree()

# Make predictions
y_pred = clf_tree.predict(X_test)

# Evaluate
accuracy = evaluate_classification(y_test, y_pred)
print(f"Iris Classification Accuracy: {accuracy:.4f}")


Decision Tree Structure:
Feature 2 <= 1.9
|-- Left: Leaf: 0
|-- Right: Feature 2 <= 4.7
|-- Right: |-- Left: Feature 3 <= 1.6
|-- Right: |-- Left: |-- Left: Leaf: 1
|-- Right: |-- Left: |-- Right: Leaf: 2
|-- Right: |-- Right: Feature 3 <= 1.7
|-- Right: |-- Right: |-- Left: Feature 2 <= 4.9
|-- Right: |-- Right: |-- Left: |-- Left: Leaf: 1
|-- Right: |-- Right: |-- Left: |-- Right: Feature 3 <= 1.5
|-- Right: |-- Right: |-- Left: |-- Right: |-- Left: Leaf: 2
|-- Right: |-- Right: |-- Left: |-- Right: |-- Right: Feature 0 <= 6.7
|-- Right: |-- Right: |-- Left: |-- Right: |-- Right: |-- Left: Leaf: 1
|-- Right: |-- Right: |-- Left: |-- Right: |-- Right: |-- Right: Leaf: 2
|-- Right: |-- Right: |-- Right: Feature 2 <= 4.8
|-- Right: |-- Right: |-- Right: |-- Left: Feature 0 <= 5.9
|-- Right: |-- Right: |-- Right: |-- Left: |-- Left: Leaf: 1
|-- Right: |-- Right: |-- Right: |-- Left: |-- Right: Leaf: 2
|-- Right: |-- Right: |-- Right: |-- Right: Leaf: 2
Iris Classification Accuracy: 1.000

在决策树的构建过程中，递归分枝（递归地划分数据集）是核心步骤之一。然而，为了防止过拟合以及确保树的结构合理，我们需要设定一些终止条件来决定何时停止进一步分枝。以下将结合我们之前实现的CART（分类与回归树）算法代码，详细介绍树递归分枝的终止条件及其对应的代码，并简要说明递归分枝的过程。

## 树递归分枝的终止条件

在我们的实现中，决策树的递归分枝主要有以下两个终止条件：

1. **最小样本分裂数（`min_samples_split`）**：
   - **定义**：如果一个节点中的样本数少于设定的最小样本分裂数，则不再对该节点进行分裂，直接将其作为叶节点。
   - **作用**：防止树过于细致地拟合训练数据，减少过拟合风险。

2. **最小不纯度减少（`min_impurity_decrease`）**：
   - **定义**：如果通过分裂一个节点所带来的不纯度（如基尼指数或方差）的减少量小于设定的阈值，则不进行分裂，直接将该节点作为叶节点。
   - **作用**：确保每次分裂都有足够的意义，避免无效分裂。


```
def _build_tree(self, X, y):
    """
    递归地构建树。
    """
    num_samples, num_features = X.shape

    # 终止条件1：样本数小于最小分裂数
    if num_samples < self.min_samples_split:
        leaf_value = self._calculate_leaf_value(y)
        return Node(value=leaf_value)

    # 寻找最佳分裂
    best_feature, best_threshold, best_gain = self._best_split(X, y, num_features)

    # 终止条件2：不纯度减少量小于阈值
    if best_gain < self.min_impurity_decrease:
        leaf_value = self._calculate_leaf_value(y)
        return Node(value=leaf_value)

    # 分裂节点
    left_indices = X[:, best_feature] <= best_threshold
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[~left_indices], y[~left_indices]

    # 递归构建左子树和右子树
    left_child = self._build_tree(X_left, y_left)
    right_child = self._build_tree(X_right, y_right)

    return Node(feature_index=best_feature, threshold=best_threshold, left=left_child, right=right_child)
```

## 树的递归分枝过程简述

结合上述代码，树的递归分枝过程可以分为以下步骤：

1. **初始化**：
   - 从根节点开始，传入整个训练数据集`X`和标签`y`。

2. **检查终止条件1（最小样本分裂数）**：
   - 如果当前节点的样本数小于`min_samples_split`，则停止分裂，创建叶节点。

3. **寻找最佳分裂**：
   - 遍历所有特征和可能的阈值，计算每个分裂的收益。
   - 选择收益最大的特征和阈值作为当前节点的分裂依据。

4. **检查终止条件2（最小不纯度减少）**：
   - 如果最佳分裂的收益小于`min_impurity_decrease`，则停止分裂，创建叶节点。

5. **进行分裂**：
   - 根据最佳特征和阈值，将数据集划分为左子集和右子集。

6. **递归构建子树**：
   - 对左子集和右子集分别调用`_build_tree`方法，递归地构建左子树和右子树。

7. **终止递归**：
   - 当所有叶节点都满足终止条件，不再进行分裂时，递归过程结束，决策树构建完成。

通过上述递归过程，决策树逐层分裂数据，直到满足预设的终止条件，从而形成一个能够进行分类或回归预测的树模型。

